# UCI SEMICOM dataset


After doing research about the dataset which can be found in the *word document* I will have put in the same folder as this analysis, I will now start to work on the dataset. I've taken a look into the dataset ( which you can also see in the sample ) and I know I have many columns with numerical variables. 

#### First we import all the important stuff and our dataset. 

In [28]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold

SemiCom = pd.read_csv("uci-secom.csv")
np.random.seed(0)

#### I will also add a function that makes sure the output is shown on full screen and not in a scrollable block.

In [29]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

<IPython.core.display.Javascript object>

***
Here we can check our data really quick.
***

In [30]:
SemiCom.sample(5)

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
319,2008-08-20 01:42:00,2926.40,2421.45,2180.9666,998.4939,1.3990,100.0,106.3311,0.1216,1.3899,...,NaN,0.5036,0.0150,0.0035,2.9800,0.0262,0.0076,0.0022,29.1606,-1
1045,2008-09-23 22:25:00,2935.34,2551.27,2276.4556,2148.5397,1.2317,100.0,93.6778,0.1199,1.4132,...,NaN,0.4941,0.0178,0.0043,3.6018,0.0257,0.0174,0.0060,67.6124,-1
1323,2008-05-10 14:34:00,3023.56,2555.88,2205.2555,1412.7131,0.9785,100.0,95.2556,0.1216,1.5407,...,NaN,0.5018,0.0127,0.0034,2.5350,0.0230,0.0210,0.0053,91.4264,-1
579,2008-08-30 11:12:00,3042.36,2493.85,2124.8444,1180.2820,0.8465,100.0,100.7978,0.1257,1.3725,...,NaN,0.4970,0.0143,0.0031,2.8703,0.0182,0.0139,0.0039,76.6094,-1
414,2008-08-22 10:47:00,2965.48,2467.94,2178.6889,1657.3518,1.6603,100.0,100.8022,0.1229,1.5611,...,43.1616,0.5008,0.0109,0.0029,2.1758,0.0433,0.0187,0.0056,43.1616,-1


### Cleaning the data

***
Before I can start to work with this dataset i need to clean it. The information from the dataset said we did have missing values so lets start to work on those:

* I first want to see what I'm dealing with. So I can decide if i want to remove columns or add values.
***

In [31]:
#Let's check how many rows and columns we have in this dataset
totaldata = np.product(SemiCom.shape)
totaldata

927664

In [32]:
#Total amount of missimg data
missingdata = SemiCom.isnull().sum()
totalmissingdata = missingdata.sum()
totalmissingdata 

41951

***
Now one thing i want to do is check the percentage of the total missing values in this dataset.
***

In [33]:
(totalmissingdata/totaldata) * 100

4.522219251798065

***
As you can see not much of the data is missing so removing these wont have a big impact since the dataset has very many values. But, it is necessary to have a clean dataset so that our prediction is more accurate. So my plan is to make a threshold of 25%. When a column is missing more then 25% the collumn gets removed.
***

In [34]:
#I wanted to check which colums had the most NaN values
missingdata.sort_values(ascending=False).head(10)

158    1429
292    1429
293    1429
157    1429
85     1341
492    1341
220    1341
358    1341
517    1018
245    1018
dtype: int64

***
So here we can see all the columns that are above the threshold and need to be removed. My next step is dropping these columns and checking before and after if columns have been removed. I wanted to do this bit with the 'dropna()' function but this drops rows or columns based on missing values. It cannot be used to drop columns that you specify.
***

In [35]:
threshold = 0.15

columns_to_drop = missingdata[missingdata > threshold * len(SemiCom)].index
print(columns_to_drop)

Index(['72', '73', '85', '109', '110', '111', '112', '157', '158', '220',
       '244', '245', '246', '247', '292', '293', '345', '346', '358', '382',
       '383', '384', '385', '492', '516', '517', '518', '519', '546', '547',
       '548', '549', '550', '551', '552', '553', '554', '555', '556', '557',
       '562', '563', '564', '565', '566', '567', '568', '569', '578', '579',
       '580', '581'],
      dtype='object')


In [36]:
print(SemiCom.columns)

Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '581', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=592)


In [37]:

SemiCom_dropped = SemiCom.drop(columns=columns_to_drop)

print(SemiCom_dropped.columns)

Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '577', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=540)


***

As you can see our columns length has gone down from 592 to 540. Now we need look for other ways to remove columns that are useless because now we still have too many columns. 
After doing some research and asking ChatGPT how i could clean a dataset that has many numerical columns. I found the Variance threshold which means that you remove the columns that have mostly the same information. And because it's almost constantly the same it is not very usefull.

This is usefull for me because my dataset has many columns with probably the same information. Which wont provide any extra information for the model.

***

In [38]:
#had a small error about the time not being able to covert to float
SemiCom_dropped['Time'] = pd.to_numeric(SemiCom_dropped['Time'], errors='coerce')


In [39]:
print(SemiCom_dropped.dtypes)

Time         float64
0            float64
1            float64
2            float64
3            float64
              ...   
586          float64
587          float64
588          float64
589          float64
Pass/Fail      int64
Length: 540, dtype: object


In [45]:

thresholder = VarianceThreshold(threshold=0.05)

X_high_variance = thresholder.fit_transform(SemiCom_dropped)
#put the remaining columns in a list
selected_features = SemiCom_dropped.columns[thresholder.get_support()].tolist()
SemiCom_filtered = SemiCom_dropped[selected_features]

SemiCom_filtered.columns

/Users/alex/opt/anaconda3/envs/DataAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


Index(['0', '1', '2', '3', '4', '6', '12', '14', '15', '16',
       ...
       '570', '571', '572', '573', '574', '576', '577', '585', '589',
       'Pass/Fail'],
      dtype='object', length=251)

***
Yep! that was a good one. We just cut our columns in half from 540 to 251.

***

In [41]:
SemiCom_filled = SemiCom_dropped.fillna(SemiCom_dropped.mean())
missingdata = SemiCom_filled.isnull().sum()
totalmissingdata = missingdata.sum()
print(totalmissingdata)

1567
